# 2. MNIST데이터셋에 대하여 SVM학습하기
svm은 이진 분류기라서 OvR전략을 사용해 10개의 숫자를 분류함  
처리속도를 높이기 위해 작은 검증 세트로 하이퍼파라미터를 조절하는 것이 좋다고 함  
가능한 높은 정확도를 달성할 것

In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784", version=1)
x, y = mnist["data"], mnist["target"]
x_train, x_test = x[: 60000], x[60000: ]
y_train, y_test = y[: 60000], y[60000: ]

In [2]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.8, random_state=42)
for mini_train_index, mini_test_index in split.split(x_train, y_train):
    print("\n---")
    print(len(mini_train_index), len(mini_test_index))
    mini_x_train = x_train.iloc[mini_train_index]
    mini_y_train = y_train.iloc[mini_train_index]
    mini_x_test = x_train.iloc[mini_test_index]
    mini_y_test = y_train.iloc[mini_test_index]


---
12000 48000


mnist데이터셋은 이미 잘 섞여 있으니 단순 인덱스로 나누어 작은 검증 세트를 뽑아낼 수 있겠지만  
일반화를 위해 **계층적 분리 스플릿** 을 이용

In [39]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC())
])

In [33]:
from scipy.ndimage.interpolation import shift
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

def shift_img(flatted_img):
    img = flatted_img.reshape((28, 28))
    dx, dy = np.random.randint(0, 3, size=2)
    shifted_img = shift(img, [dy, dx], cval=0, mode="constant")
    return shifted_img.reshape([-1])

class ImgShifter(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = np.apply_along_axis(shift_img, -1, X)
        return X

In [34]:
preprocessing_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("shfter", ImgShifter())
])

preprocessing_pipeline.fit_transform(mini_x_train)

(12000, 784)


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -3.31284797e-02, -1.37977840e-02, -9.12908968e-03],
       [ 0.00000000e+00,  7.02995108e-21, -9.46818715e-21, ...,
        -9.27533742e-19, -8.47884556e-19, -9.89933632e-19],
       [ 2.53212601e-22, -2.69015966e-21, -1.11546898e-19, ...,
        -8.20683076e-19, -7.73286048e-19,  2.47295715e-19],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -3.31284797e-02, -1.37977840e-02, -9.12908968e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -9.12908968e-03, -9.12908968e-03, -2.49681380e-18],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -9.12908968e-03, -4.84343112e-19,  1.30292847e-19]])

mnist 이미지를 **쉬프트한 데이터로 증강하면** 더 좋은 정확도를 얻을 수 있기 때문에 전처리 파이프라인으로 생성

In [36]:
'''svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("shifter", ImgShifter()),
    ("svm_clf", SVC())
])'''

In [37]:
svm_clf.fit(mini_x_train, mini_y_train)

(12000, 784)


Pipeline(steps=[('scaler', StandardScaler()), ('shifter', ImgShifter()),
                ('svm_clf', SVC())])

In [38]:
from sklearn.metrics import accuracy_score
svm_predict = svm_clf.predict(mini_x_test)
accuracy_score(mini_y_test, svm_predict)

(48000, 784)


0.914125

???? shifter를 추가하기 전에는 94%였는데 더 떨어짐(????)  
대체 뭐지...  
그래서 아래 그리드탐색은 std 스케일링만 한걸로 진행함

In [40]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {"svm_clf__C": [0.01, 0.1, 1, 10, 100], 
    "svm_clf__kernel": ["linear", "poly", "rbf"],
    "svm_clf__gamma": ["scale", "auto"]}
]

grid_search = GridSearchCV(svm_clf, param_grid, cv=5, 
                          scoring="accuracy",
                          return_train_score=True)

In [41]:
grid_search.fit(mini_x_train, mini_y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svm_clf', SVC())]),
             param_grid=[{'svm_clf__C': [0.01, 0.1, 1, 10, 100],
                          'svm_clf__gamma': ['scale', 'auto'],
                          'svm_clf__kernel': ['linear', 'poly', 'rbf']}],
             return_train_score=True, scoring='accuracy')

In [42]:
grid_search.best_params_

{'svm_clf__C': 100, 'svm_clf__gamma': 'scale', 'svm_clf__kernel': 'poly'}

In [43]:
best_svm = grid_search.best_estimator_

In [44]:
best_predict = best_svm.predict(mini_x_test)
accuracy_score(mini_y_test, best_predict)

0.964

확실히 그리드 탐색을 하고 파라미터를 수정하니 성능이 올라감  
학습데이터를 증가시키면 성능이 더 좋을듯  
> 일단 결과를 보면 이전에 **KNN**분류기를 사용했을 때보다 걸린 시간, 성능 둘 다 떨어짐...  
데이터셋에 따라 다른건가??